In [ ]:
import canvasapi

In [ ]:
API_URL = "https://uweau.instructure.com/"
from canvas_credentials import API_KEY  # i put my key in another file.  
canvas = canvasapi.Canvas(API_URL, API_KEY)

In [ ]:
#get from the URL for your course.  you could also find it programmatically...
calc1_fall_2020 = 310046
sandbox = 127210000000000267

course_id = calc1_fall_2020

In [ ]:
course = canvas.get_course(course_id) 
E = course.get_enrollments()
user_ids_list = [ [e.user_id, e.user["name"], int(e.sis_section_id.split("-")[4])] for e in E if (e.user["name"]!= "Silviana Amethyst" and e.user["name"]!= "Test Student")] # the 002 or 004 for their section is the [4] entry in this split list

In [ ]:
# turn the list into a dict for looking up students by `user_id`
students_by_id = {}
for u in user_ids_list:
    students_by_id[u[0]] = {'name':u[1],'section':u[2]} 

# i am sure this cell could be elided by using a dict comprehension instead of a list comprehension in the preceding cell.

In [ ]:
students_by_name = {}
for u in user_ids_list:
    students_by_name[u[1]] = {'id':u[0],'section':u[2]} 

In [ ]:
groups = course.get_groups() # this runs lazily, just so you know.  they use pagination.  see their documentation: https://github.com/ucfopen/canvasapi

In [ ]:
# find the mismatched groups
found_misgrouped_students = False
for g in groups:
    if "group" in g.name:  #i had several sets of groups, and the ones i needed had "group" in their name, and the ones I didn't care about didn't
        section_number = int(g.name.split(',')[1].split()[1]) # parse section number from group name.  again, i had put them there
        members = g.get_memberships()
        for u in members:  # for each student in the group
            if students_by_id[u.user_id]['section']!=section_number:
                print(g.name)
                print(students_by_id[u.user_id])
                found_misgrouped_students = True
if not found_misgrouped_students:
    print('found no misgrouped students, huzzah')
# note: this solutions is a bit convoluted, perhaps.
# i wanted to call `canvas.get_user(u.user_id)`, and get their enrollments.  
# that was the most obvious solution, but it didn't work for me.
# reason: i do not have permissions to call that function.
# but, you *can* get_current_user(), and get yourself!

In [ ]:
# find the sections of ungrouped students.  
# this will always include me, and the test student.
enrolled_students = set([v['id'] for e,v in students_by_name.items()])
grouped_students = []
for g in groups:
    for u in g.get_memberships():
        grouped_students.append(u.user_id)
grouped_students = set(grouped_students)

ungrouped_students = enrolled_students - grouped_students
for s in ungrouped_students:
    print(students_by_id[s])
if len(ungrouped_students)==0:
    print("no ungrouped students")

In [ ]:
# look up a student by name to see which groups they're in
# it appears to use First Last as the name format.  
student_name = "First Last"
student_id = students_by_name[student_name]['id']
for g in groups:
    members = g.get_memberships()
    for m in members:
        if m.user_id == student_id:
            print('{} is in group {}'.format(student_name,g.name))